In [1]:
%pip install opencv-python

In [2]:
%pip install lpips

Note: you may need to restart the kernel to use updated packages.


In [3]:
!python --version

Python 3.12.4


In [4]:
import cv2

import torch
import numpy as np

import os

## Metrics set-up

In [5]:
%pip install SSIM-PIL

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install torchmetrics

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install sewar

Note: you may need to restart the kernel to use updated packages.


In [8]:
from scipy import signal

# code from https://github.com/BiswarupDebnath/SSIM-Map-calculation-of-an-image-in-Python/blob/master/ssim_map.py
def cal_ssim(img1, img2):

    K = [0.01, 0.03]
    L = 255
    kernelX = cv2.getGaussianKernel(11, 1.5)
    window = kernelX * kernelX.T

    M,N = np.shape(img1)

    C1 = (K[0]*L)**2
    C2 = (K[1]*L)**2
    img1 = np.float64(img1)
    img2 = np.float64(img2)

    mu1 = signal.convolve2d(img1, window, 'valid')
    mu2 = signal.convolve2d(img2, window, 'valid')

    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2


    sigma1_sq = signal.convolve2d(img1*img1, window, 'valid') - mu1_sq
    sigma2_sq = signal.convolve2d(img2*img2, window, 'valid') - mu2_sq
    sigma12 = signal.convolve2d(img1*img2, window, 'valid') - mu1_mu2

    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))
    mssim = np.mean(ssim_map)
    return mssim,ssim_map

### Image loading as required by different libraries

In [9]:
from PIL import Image

def load_openCV_format(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image could not be loaded.")
    # img = cv2.resize(img, size) # images must be of the same size to be compared for eg ssim
    return img


def load_pil_object_img(image_path):
    return Image.open(image_path)


In [10]:
from skimage.metrics import structural_similarity as ssim_sklearn
import lpips
from SSIM_PIL import compare_ssim
from torchmetrics.image import UniversalImageQualityIndex
from torchmetrics.image import VisualInformationFidelity
from sewar import uqi, vifp, ssim



def get_scores(reference_img_path, distorted_img_path):
    ref_img = load_openCV_format(reference_img_path)
    distorted_img = load_openCV_format(distorted_img_path)

    # ssim_github_code_score = cal_ssim(cv2.cvtColor(ref_img, cv2.COLOR_BGR2RGB), cv2.cvtColor(distorted_img, cv2.COLOR_BGR2RGB))

    # ref_gray = cv2.cvtColor(ref_img, cv2.COLOR_BGR2GRAY) 
    # distorted_gray = cv2.cvtColor(distorted_img, cv2.COLOR_BGR2GRAY)

    ssim_sklearn_score, _ = ssim_sklearn(ref_img, distorted_img, full=True, channel_axis=2)
    psnr_score = cv2.PSNR(ref_img, distorted_img) 

    
    normalized_ref_img = (ref_img.astype(np.float32) / 127.5) - 1.0 
    normalized_distorted_img = (distorted_img.astype(np.float32) / 127.5) - 1.0

    tensor_ref_img = torch.from_numpy(normalized_ref_img).permute(2, 0, 1)
    tensor_distorted_img = torch.from_numpy(normalized_distorted_img).permute(2, 0, 1)

    loss_fn_alex = lpips.LPIPS(net='alex')
    lpips_score = loss_fn_alex(tensor_ref_img, tensor_distorted_img).item()

    uiqi_torch = UniversalImageQualityIndex()
    uiqi_torch_score = uiqi_torch(tensor_ref_img.expand(1,-1,-1,-1), tensor_distorted_img.expand(1,-1,-1,-1))

    vif_torch = VisualInformationFidelity()
    vif_torch_score = vif_torch(tensor_ref_img.expand(1,-1,-1,-1), tensor_distorted_img.expand(1,-1,-1,-1))


    # sewar lib 
    pil_ref_img = load_pil_object_img(reference_img_path)
    pil_distorted_img = load_pil_object_img(distorted_img_path)

    ref_yc = np.array(pil_ref_img.convert('YCbCr'))
    distorted_yc = np.array(pil_distorted_img.convert('YCbCr'))

    uiqi_sewar_score = uqi(ref_yc, distorted_yc)
    ssim_sewar_score = ssim(ref_yc, distorted_yc)
    vifp_sewar_score = vifp(ref_yc, distorted_yc)

    ssim_pil = compare_ssim(pil_ref_img, pil_distorted_img)

    

    return ssim_sklearn_score,ssim_sewar_score, ssim_pil, psnr_score, lpips_score, uiqi_torch_score.item(), vif_torch_score.item(), uiqi_sewar_score, vifp_sewar_score


    


c:\Python312\Lib\site-packages\pyopencl\cache.py:496: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  _create_built_program_from_source_cached(


In [11]:
%pip install pandas 
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
import pandas as pd

def save_results(results, path):
    root_name = os.path.basename(os.path.normpath(path))
    filename = f"{root_name}.xlsx"
    column_names = [
        "ssim_sklearn_score", "ssim_sewar_score", "ssim_pil", "psnr_score", "lpips_score",
        "uiqi_torch_score", "vif_torch_score", "uiqi_sewar_score", "vifp_sewar_score"
    ]

    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        for subfolder_name, result_list in results:
            # Ensure each result is a list (in case it’s a tuple or other type)
            df = pd.DataFrame([list(scores) for scores in result_list], columns=column_names)
            df.to_excel(writer, sheet_name=subfolder_name[:31], index=False)  # Excel sheet names max 31 chars

    print(f"Results saved to {filename}")

In [13]:
def parse_images(root_path):
    subfolders = [f.path for f in os.scandir(root_path) if f.is_dir()]
    results = []
    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        image_paths = [os.path.join(subfolder, fname) for fname in os.listdir(subfolder)]

        subfolder_results = []
        for i in range(len(image_paths)):
            scores = get_scores(image_paths[0], image_paths[i])
            subfolder_results.append(scores)
        results.append((subfolder_name, subfolder_results))

    save_results(results, root_path)
        

In [14]:
img1 = 'image_1st_in_sequence.jpg'
img2 = 'image_2nd_in_sequence.jpg'

print(get_scores(img1, img2))

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


C:\Users\piotr\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\piotr\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Users\piotr\AppData\Roaming\Python\Python312\site-packages\lpips\weights\v0.1\alex.pth
(np.float64(0.12952294418603907), (np.float64(0.646707449310005), np.float64(0.6477909385615072)), np.float32(0.13061553), 15.3074458157371, 0.42474567890167236, 0.005599730648100376, 0.02949672006070614, np.float64(0.9777349890404293), np.float64(0.05249949081500604))


In [15]:
root_folder_path = 'C:/Users/piotr/Desktop/Projekt Wkiro/image_similarity-main-v2/Photos_small/Wall'
results = parse_images(root_folder_path)

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\piotr\AppData\Roaming\Python\Python312\site-packages\lpips\weights\v0.1\alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\piotr\AppData\Roaming\Python\Python312\site-packages\lpips\weights\v0.1\alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\piotr\AppData\Roaming\Python\Python312\site-packages\lpips\weights\v0.1\alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\piotr\AppData\Roaming\Python\Python312\site-packages\lpips\weights\v0.1\alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\piotr\AppData\Roaming\Python\Python312\site-packages\lpips\weights\v0.1\alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from